In [ ]:
import os

import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [ ]:
# Create a data frame from the college scorecard cohorts-institutions CSV file.

institutions_filename = '../data/Most-Recent-Cohorts-Institution.csv.gz'
institutions_df = pd.read_csv(institutions_filename,
                usecols=['OPEID6', 
                         'INSTNM', 'CITY', 'STABBR', 
                         'FTFTPCTPELL', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'ADM_RATE', 
                         'NPT4_PUB', 'NPT4_PRIV',
                         'NPT41_PUB', 'NPT41_PRIV',
                         'NPT45_PUB', 'NPT45_PRIV', 
                         'MD_EARN_WNE_P10', 'C100_4'])

In [ ]:
institutions_df.head()

In [ ]:
# Load the CSV file for fields of study into another data frame

fields_filename = '../data/FieldOfStudyData1718_1819_PP.csv.gz'

fields_of_study_df = pd.read_csv(fields_filename,
                                usecols=['OPEID6', 'INSTNM', 'CREDDESC', 'CIPDESC', 'CONTROL'])

In [ ]:
fields_of_study_df.head()

In [ ]:
# What state has the greatest number of universities in this database?

(
    institutions_df
    .groupby('STABBR')['OPEID6'].count()
    .sort_values(ascending=False)
    .head(1)
)    

In [ ]:
# What city+state has the greatest number of universities in this database?

(
    institutions_df
    .groupby(['STABBR', 'CITY'])['OPEID6'].count()
    .sort_values(ascending=False)
    .head(1)
)

In [ ]:
# How much memory can we save if we set the CITY and STABBR columns in institutions_df to be categories?
pre_category_memory = (
    institutions_df
    .memory_usage(deep=True)
    .sum()
)

print(f'{pre_category_memory:,}')

In [ ]:
institutions_df['CITY'] = (
    institutions_df['CITY']
    .astype('category')
)

institutions_df['STABBR'] = (
    institutions_df['STABBR']
    .astype('category')
)

post_category_memory = (
    institutions_df
    .memory_usage(deep=True)
    .sum()
)

savings = pre_category_memory - post_category_memory
print(f'{savings:,}')

In [ ]:
# Create a histogram showing how many bachelor programs universities offer
p = (
    fields_of_study_df
    .loc[fields_of_study_df['CREDDESC'] == 'Bachelors Degree']
    .groupby('INSTNM')['CIPDESC'].count()
    .plot.hist()
)

f = p.get_figure()
f.savefig('/tmp/CH12_F1_LERNER.jpg')

In [ ]:
# Which university offers the greatest number of bachelor programs?
(
    fields_of_study_df
    .loc[fields_of_study_df['CREDDESC'] == 'Bachelors Degree']
    .groupby('INSTNM')['CIPDESC'].count()
    .sort_values(ascending=False)
    .head(10)
)

In [ ]:
# Create a histogram showing how many graduate (master's and doctoral) programs universities offer
p = (
    fields_of_study_df
    .loc[fields_of_study_df['CREDDESC']
          .isin(["Master's Degree", "Doctoral Degree"])]
    .groupby('INSTNM')['CIPDESC'].count()
    .plot.hist()
)

f = p.get_figure()
f.savefig('/tmp/CH12_F2_LERNER.jpg')

In [ ]:
# Which university offers the greatest number of different graduate (master + doctoral) programs?

(
    fields_of_study_df
    .loc[fields_of_study_df['CREDDESC']
        .isin(["Master's Degree", "Doctoral Degree"])]
    .groupby('INSTNM')['CIPDESC'].count()
    .sort_values(ascending=False)
    .head(10)
)

In [ ]:
# How many universities offer bachelor's degrees, but not master's or doctorates?

ug_schools = (
    fields_of_study_df
    .loc[fields_of_study_df['CREDDESC'] == 'Bachelors Degree',
    'INSTNM']
)

grad_schools = (
    fields_of_study_df
    .loc[fields_of_study_df['CREDDESC']
        .isin(["Master's Degree", "Doctoral Degree"]), 
    'INSTNM']
)

ug_schools[~ug_schools.isin(grad_schools)].drop_duplicates().size

In [ ]:
# How many universities offer master's and doctoral degrees, but not bachelors?
grad_schools[~grad_schools.isin(ug_schools)].drop_duplicates().size

In [ ]:
# How many institutions offer bachelor's degrees whose name contains the term "Computer Science"?

(
    fields_of_study_df.loc[(fields_of_study_df['CIPDESC']
                        .str.contains('Computer Science')) &
                       (fields_of_study_df['CREDDESC'] == 'Bachelors Degree'),
                       'INSTNM']
    .unique()
    .size
)

In [ ]:
# The CONTROL field describes the types of institutions in the database.
# How many of each type offer a computer-science program?

(
    fields_of_study_df
    .loc[(fields_of_study_df['CIPDESC']
            .str.contains('Computer Science')) &
         (fields_of_study_df['CREDDESC'] == 'Bachelors Degree'),
    ['CONTROL','INSTNM']].groupby('CONTROL').count()
)

In [ ]:
# Create a pie chart showing the different types of institutions that offer CS degrees
p = (
    fields_of_study_df
    .loc[(fields_of_study_df['CIPDESC']
              .str.contains('Computer Science')) &
         (fields_of_study_df['CREDDESC'] == 'Bachelors Degree'),
    ['CONTROL','INSTNM']]
    .groupby('CONTROL').count()['INSTNM']
    .plot.pie()
)

f = p.get_figure()
f.savefig('/tmp/CH12_F3_LERNER.jpg')

In [ ]:
# What are the minimum, median, mean, and maximum tuitions for an undergrad CS degree?  
# (We'll define this as a bachelor's program with the phrase "Computer Science" in the name.)
# When comparing tuition, use TUITIONFEE_OUT for all schools.

comp_sci_universities = (
    fields_of_study_df
    .loc[(fields_of_study_df['CIPDESC']
            .str.contains('Computer Science')) &
         (fields_of_study_df['CREDDESC'] == 'Bachelors Degree'), 
    ['OPEID6','CONTROL','INSTNM']]
    .set_index('OPEID6')
)

(
    comp_sci_universities
    .join(institutions_df[['OPEID6', 'TUITIONFEE_OUT']]
          .set_index('OPEID6'))
    ['TUITIONFEE_OUT']
    .describe()
)

In [ ]:
# Describe the tuition again, but grouped by the different types of universities ("CONTROL")

comp_sci_universities = (
    fields_of_study_df
    .loc[(fields_of_study_df['CIPDESC']
              .str.contains('Computer Science')) &
         (fields_of_study_df['CREDDESC'] == 'Bachelors Degree'),
    ['OPEID6','CONTROL','INSTNM']]
    .set_index('OPEID6')
)

(
    comp_sci_universities
    .join(institutions_df[['OPEID6', 'TUITIONFEE_OUT']]
          .set_index('OPEID6'))
    .groupby('CONTROL')['TUITIONFEE_OUT'].describe()
    .dropna()
    .T
)

In [ ]:
# What is the correlation between admission rate and tuition cost?
# How would you interpret this?

institutions_df[['ADM_RATE', 'TUITIONFEE_OUT']].corr()

In [ ]:
# Create a scatter plot in which tuition is on the x axis, and admission rate is on the y axis,
# the median earnings after 10 years are used for colorizing, and we use the "Spectral" colormap.
# Where do the lowest-paid graduates show up on the graph?

p = (
    institutions_df
    .plot.scatter(x='TUITIONFEE_OUT', 
                  y='ADM_RATE', 
                  c='MD_EARN_WNE_P10', 
                  colormap='Spectral')
)

f = p.get_figure()
f.savefig('/tmp/CH12_F4_LERNER.jpg')

In [ ]:
# Which universities are in the top 25% of tuition, and also the top 25% of percentage with Pell grants?
# Print only the institution name, city, and state, ordered by institution name

(
    institutions_df
    .loc[(institutions_df['TUITIONFEE_OUT'] >
          institutions_df['TUITIONFEE_OUT'].quantile(0.75)) &
         (institutions_df['FTFTPCTPELL'] >
          institutions_df['FTFTPCTPELL'].quantile(0.75)),
        ['INSTNM', 'CITY', 'STABBR']]
    .sort_values(by='INSTNM')
)

In [ ]:
# NPT4_PUB indicates the average net price for public institutions (in-state tuition)
# and NPT4_PRIV for private institutions.  

# NPT41_PUB and NPT45_PUB show the average price paid by people in the lowest income
# bracket (1) vs. the highest income bracket (5) at public institutions.

# NPT41_PRIV and NPT45_PRIV show the average price paid by people in the lowest income
# bracket (1) vs. the highest income bracket (5) at private institutions.

# In how many institutions does the bottom quintile receive money (i.e., is the 
# value negative)?

# Using logical "or" with |

(
    institutions_df
    .loc[((institutions_df['NPT41_PUB'] < 0) |
          (institutions_df['NPT41_PRIV'] < 0)), 
    'INSTNM']
    .count()
)

In [ ]:
# Using .add
institutions_df.loc[institutions_df['NPT41_PUB'].add(institutions_df['NPT41_PRIV'], fill_value=0) < 0,
                   'INSTNM'].count()

In [ ]:
# What is the average proportion that the bottom quintile pays vs. the top quintile, in public universities?

(
    (institutions_df['NPT41_PUB'] 
     / institutions_df['NPT45_PUB'])
    .mean()
)

In [ ]:
# What is the average proportion that the bottom quintile pays vs. the top quintile, in private universities?

(
    (institutions_df['NPT41_PRIV'] 
     / institutions_df['NPT45_PRIV'])
    .mean()
)

In [ ]:
# Let's try to figure out which universities offer the best overall ROI (across all disciplines).
# What schools are in the cheapest 25%, but 10 years after graduation, students have the top 25% of salaries?


# First, public institutions
(
    institutions_df
    .loc[(institutions_df['NPT4_PUB'] 
          <= institutions_df['NPT4_PUB'].quantile(0.25)) &
         (institutions_df['MD_EARN_WNE_P10'] 
          >= institutions_df['MD_EARN_WNE_P10'].quantile(0.75)),
        ['INSTNM', 'STABBR', 'CITY']]
    .sort_values(by=['STABBR', 'CITY'])
)

In [ ]:
# How about private institutions?

(
    institutions_df
    .loc[(institutions_df['NPT4_PRIV'] 
          <= institutions_df['NPT4_PRIV'].quantile(0.25)) &
         (institutions_df['MD_EARN_WNE_P10']
          >= institutions_df['MD_EARN_WNE_P10'].quantile(0.75)),
    ['INSTNM', 'STABBR', 'CITY']]
    .sort_values(by=['STABBR', 'CITY'])
)

In [ ]:
# Is there a correlation between admission rates and completion rates?  If a school is highly
# selective, are students more likely to graduate?

institutions_df[['C100_4', 'ADM_RATE']].corr()

In [ ]:
# Ten years after graduating, from what kinds of schools (private, for-profit, private non-profit,
# or public) do people earn, on average, the greatest amount?

(
    institutions_df[['OPEID6', 'MD_EARN_WNE_P10']]
    .set_index('OPEID6')
    .join(fields_of_study_df
          .groupby('OPEID6')['CONTROL'].min())
    .groupby('CONTROL')
    .mean()
)

In [ ]:
# Do people who graduate from "Ivy Plus" schools earn more than the average private-school
# university graduate?  If so, then how much more?

ivy_plus = ['Harvard University', 
            'Massachusetts Institute of Technology',
            'Yale University',
            'Columbia University in the City of New York',
            'Brown University',
            'Stanford University',
            'University of Chicago',
            'Dartmouth College',
            'University of Pennsylvania',
            'Cornell University',
            'Princeton University']

(
    institutions_df
    .loc[institutions_df['INSTNM'].isin(ivy_plus), 
         'MD_EARN_WNE_P10']
    .mean()
)

In [ ]:
# Do people studying at universities in particular states earn, on average, more 
# after 10 years?

(
    institutions_df
    .groupby('STABBR', observed=True)['MD_EARN_WNE_P10'].mean()
    .sort_values(ascending=False)
)

In [ ]:
# Create a bar plot for the average amount earned, per state, sorted by ascending pay

p = (
    institutions_df
    .groupby('STABBR', observed=True)
    ['MD_EARN_WNE_P10'].mean()
    .sort_values()
    .plot.bar(figsize=(20,10))
)

f = p.get_figure()
f.savefig('/tmp/CH12_F5_LERNER.jpg')

In [ ]:
# Create a boxplot for the earnings by state.

p = (
    institutions_df
    .groupby('STABBR', observed=True)
    ['MD_EARN_WNE_P10'].mean()
    .plot.box()
)

f = p.get_figure()
f.savefig('/tmp/CH12_F6_LERNER.jpg')